### Get structure for factors-patients graph

1. Transform all factors to boolean form using quartiles and one-hot categorisation algorithm. Save csv with index of case ID.

In [1]:
#Импорты библиотек и вспомогательных функций
%run C:\Users\Илья\Desktop\Work\support_functions.ipynb

Automatically created module for IPython interactive environment


In [2]:
equal_intetvent_names=[['relaxation techniques:jacobson relaxation',
                        'relaxation methods:relaxation according to jacobson'],
                       ["life skills:activity development/maintenance",
                        "everyday skills:activity development/maintenance"],
                       ["n/a:therapy in an individual setting",'individual:n/a'],
                      ]
#Агрегируем одинаковые названия вмешательств
def aggregate_interventions(df):
    for names in equal_intetvent_names:
        print(df[names[0]].value_counts())
        df[names[0]]=pd.Series(list(zip(*[df[n].to_list() for n in names])),index=df.index).apply(max)
        df.drop(names[1:],axis=1,inplace=True)
        print(df[names[0]].value_counts())

In [3]:
rename_dict={'Client:Treatment/Care:Life Skills:Activity Development/Maintenance:Group:N/A':'Life Skills - actibvity development  Group - NA',
 'Client:treatment/care:everyday skills:activity development/maintenance:group:mobile':'Everyday skills - activity development  Group - mobile',
 'Client:Treatment/care:Relaxation methods:Relaxation according to Jacobson:Group:Outpatient':'Relaxation according to Jacobson  Group - Outpatient',
 'Client:Treatment/Care:Social Competence:Communication Training:Group:N/A':'Social Competence - Communication Training  Group - NA',
 'Client:Treatment/care:Therapy:Resource work:Group:mobile':"Therapy - Resource work  Group - mobile",
 'Client:Treatment/Care:Therapy:Cognitive Training:Group:N/A':'Therapy - Cognitive Training  Group - NA',
 'Client: Treatment/care: Relaxation techniques: Jacobson relaxation: Group: N/A':"Relaxation techniques - Jacobson relaxation  Group - NA",
 'Client:Treatment/Care:Therapy:Resource Work:Group:N/A':'Therapy - Resource Work Group - NA',
 'Client: Treatment/care: Therapy: Dealing with psychotic symptoms: Group: Outpatient':"Therapy - Dealing with psychotic symptoms  Group - Outpatient",
 'Client: Treatment/care: Therapy: Resource work: Group: Outpatient':'Therapy - Resource work  Group - Outpatient',
 'Client:Treatment/care:Everyday competence:Building/maintaining daily structure:Group:N/A':"Everyday competence - Building daily structure  Group - NA",
 'Client:Treatment/Care:Life Skills:Training in Health Awareness and Behavior:Group:N/A':'Life Skills - Training in Health Awareness and Behavior  Group - NA',
 'Client:Treatment/Care:Therapy:Depression Coping:Group:N/A':'Therapy - Depression Coping - Group - NA',
 'Client: Treatment/Care: De-escalation (crisis intervention): N/A: Group: N/A':'De-escalation (crisis intervention)  Group - NA',
 'Client:Treatment/Care:Psychoeducation:N/A:Group:N/A':"Psychoeducation  Group - NA",
 'Client:treatment/care:everyday skills:activity development/maintenance:group:outpatient':"Everyday skills - activity development  Group - Outpatient",
 'Client:Treatment/care:N/A:Therapy in an individual setting:Individual:N/A':"Therapy in an individual setting  Individual - NA",
 'Client:Treatment/Care:Therapy:Emotional Competence Traning:Group:N/A':'Therapy - Emotional Competence Traning  Group - NA',
 'duration_treatment':'treatment duration (months)',
 "T0_age":"age (years)",
 }

def remane_col(col):
    if col in rename_dict:
        col=rename_dict[col]
    col=col.replace("T0_","")
    #Поднимаем только 1 букву если она маленькая
#     if col[0].islower():
#         col=col.capitalize()
    if (len(col)==3)&(col not in ["Age","GAF"]):
        col=f"ICD {col}"
        
    #Чистим целевые переменные 
    col=col.replace("_"," ")
    return col

In [4]:
def generating_dataset():
    df=pd.read_pickle("Data/mental_pats_for_modeling_T0_Last_Points_simple_diagnosis_for_all_points.pkl")

#     #Вмешательства
#     interventions=[c for c in df.columns if (':' in c)&(" : " not in c)&("Client" in c)]
    
    #Отбираем вмешательства без групп
    interventions_witout_group=[c for c in df.columns if (':' in c)&(" : " not in c)&("Client" not in c)&("group" not in c)]
    
    aggregate_interventions(df)
    interventions=[c for c in df.columns if c in interventions_witout_group]
    print(interventions)
    
    #Диагнозы
    diagnosis=[c for c in df.columns if (re.match(r"[A-Z][0-9][0-9]",c)!=None)]
    
    #Отбираем диагнозы которые есть минимум у 10% пациентов
    diagnosis=[c for c in diagnosis if df[c].mean()>0.025]
    
    # #Отбираем пациентов у которых есть минимум 1 из этих диагнозов
    df=df[pd.Series(list(zip(*[df[c] for c in diagnosis])),index=df.index).apply(sum)>0]

    #Социальные признаки
    social_features=["T0_age",'T0_gender : women','T0_gender : man']+["duration_treatment"]#+[c for c in df.columns if "T0_referrer" in c]
    df["T0_age"]=df["T0_age"].astype(int)
    
    #Goal features на момент T0
    goal_features=["GAF",'BSI18_Somatization','BSI18_Depression',"BSI18_Anxiety","ICF13_mean",
                   "WQO_physical_health", "WQO_psychological", 'WQO_social_relationships', "WQO_environment", "WQO_quality_of_life",
                   'WQO_health_status',"WQOLBRX2"]
    
    goal_features_T0=[f"T0_{c}" for c in goal_features]
    
    outcome_features_with_many_gaps=['Progress_GAF',
                                     'Progress_BSI18_Somatization',
                                     'Progress_BSI18_Depression',
                                     'Progress_BSI18_Anxiety']
    not_outcome=['Progress_WQOLBRX2']
    
    #Целевые переменные
#     outcome_features=[c for c in df.columns if ("Progress" in c)&(c not in (outcome_features_with_many_gaps+not_outcome))]
    outcome_features=["Progress_WQO_quality_of_life"]
    
    print(len(outcome_features))
    
    print("Diag",len(diagnosis),"Intervent",len(interventions))
    fearures_for_BN=diagnosis+interventions #+social_features + outcome_features #+goal_features_T0+['cluster']
    df_BN=df[fearures_for_BN]

# #     Удаляем признаки без outcome_features
#     df_BN.dropna(subset=outcome_features,thresh=len(outcome_features),inplace=True)
    print(df_BN.shape)
    return df_BN,outcome_features

In [5]:
#Приведение датасета к виду пригодному для моделирования
def extract_table_for_graph(df,method_numerical_to_categor,disease):
    
    #Получаем категориальные переменные
    categor_feats=[f for f in df.columns if feature_is_categor(df,f,border_numbers_values=7,test_is_int=True)&("WQO" not in f)]

    #Получаем некатегориальные переменные
    numerical_feats=list(set(df.columns)-set(categor_feats))
    
    print(numerical_feats)
#     #Удаляем выбросы эвристикой межквартильного интервала
#     df=replace_emissions(df,feats_for_BN,categor_feats)

#     #Заменяем пропуски на медианы для числовых переменных
#     df=replace_passies('median',df,numerical_feats)
    
#     #Заменяем пропуски на самое частое значение в категориальных переменных
#     df=replace_passes_categor_feats(df,categor_feats)

    #Переводим все переменные из дискретного вида в непрерывный
#     with open("Data/referens.json","r") as file:
#         referens=json.load(file)
    
    
    df=numerical_to_categor(
        df=df,
        referens=[], #Не используем референсные, соответственно интервалы не нужны
        numerical_feats=numerical_feats,
        method=method_numerical_to_categor,
        to_boolean=True,
        to_interpreted_line=True
    )
    
    #Transform nebinary categor features to binary
    no_boolean_categor_feats=[f for f in df.columns if (not feature_is_categor(df,f,border_numbers_values=2))&(feature_is_categor(df,f,border_numbers_values=7))]
    
    print(no_boolean_categor_feats)
    categor_coding(df,no_boolean_categor_feats,type_coding='OneHotEncoder',drop_old_feat=True)
    
    #Extract only boolean feats plus case_ID
    boolean_categor_feats=boolean_categor_feats=[f for f in df.columns if feature_is_categor(df,f,border_numbers_values=2)]
    print("Признаки которые не попали в таблицу",set(df.columns)-set(boolean_categor_feats))
    df=df[boolean_categor_feats]
    
    #Features for drop
    for_drop=["WQOLBRX2"]
    df.columns=[remane_col(c) for c in df.columns if c not in for_drop]
    
    
    df.to_csv("Data/adjacency matrices/patienst_intervents_all_diagnosis_without_other_nodes_table_itrerv_without_groups.csv")
    return df

In [6]:
#Генерируем данные без пропусков для обучения
# df_BN,targets=generating_dataset()
# table=extract_table_for_graph(df_BN,method_numerical_to_categor="quartiles",disease="mental health")

In [7]:
# df_BN,targets=generating_dataset()

#### Calculating hemming distanse

In [66]:
path_to_df_BN="Data/adjacency matrices/patienst_intervents_all_diagnosis_without_other_nodes_table_itrerv_without_groups.csv"
vectors=pd.read_csv(path_to_df_BN,index_col="case_ID")


In [176]:
from scipy.spatial.distance import hamming

#Vectors - dataframe, columns is factors for that we wont calculate distanse
def calculating_hamming_distance(vectors,border=1,write_weight=False,save_table=False):
    hamming_dist=pd.DataFrame(columns=vectors.columns,index=vectors.columns)
    for f1 in vectors.columns:
        for f2 in vectors.columns:
            #Считаем дистанцию хамминга модифицированно, только если есть 1 в строке, 
            #т.к. иначе чем реже диагноз - тем ближе он к другим редким диагнозам, а это неправильно
#             for_hamming=vectors[(vectors[f1]==1)|(vectors[f2]==1)]
#             if for_hamming.shape[0]>10: #Минимум 10 пациентов для расчета
#                 dist=hamming(for_hamming[f1],for_hamming[f2])
#             else:
#                 dist=1
            dist=hamming(vectors[f1],vectors[f2])
            #write only dist < border
            if (dist<=border)&(f1!=f2):
                if write_weight:
                    hamming_dist[f1][f2]=1-dist
                else:
                    hamming_dist[f1][f2]=1
            else:
                hamming_dist[f1][f2]=0
                
                    
    print(f"Average {hamming_dist.mean().mean()}")
    if save_table:
        hamming_dist.to_csv(f"Data/adjacency matrices/hamming_dist_border_{border}.csv")
    return hamming_dist

# hamming_dist=calculating_hamming_distance(df_BN,border=1)

# hamming_dist

Find border for maximization equal edges for any from two BN graphs

In [165]:
from scipy.optimize import minimize

def get_edges(matrix):
    edges=[]
    for node1 in matrix:
        for node2 in matrix:
            if (matrix[node1][node2]==1)&({node1,node2} not in edges):
                edges.append({node1,node2})
    return edges


def percent_equal_edges(border,edges_other_matrix):
    hamming_dist=calculating_hamming_distance(vectors,border)
    edges_hamming=get_edges(hamming_dist)
    number_equel=0
    for edge in edges_other_matrix:
        if edge in edges_hamming:
            number_equel+=1

    if len(edges_hamming)>0:
        percent_equel=number_equel*(1.1)/len(edges_hamming)
    else:
        return 0 #Выводим максимальное значение если percent_equel==0
    return percent_equel #for minimiszation
    
def find_borders_for_optimization_numbers_of_needed_edges(other_graph_matrix_path):
    other_matrix=pd.read_csv(other_graph_matrix_path,index_col="source")
    edges_other_matrix=get_edges(other_matrix)
    print(edges_other_matrix)
    borders=list(np.arange(0.01,1,0.002))
    
    #minimizing
    optimal_border=None
    max_percent_equel=0
    for b in borders:
        percent_equal=percent_equal_edges(b,edges_other_matrix)
        if percent_equal>max_percent_equel:
            optimal_border=b
            max_percent_equel=percent_equal
    print(optimal_border,max_percent_equel)

    
def find_border_for_half_edges(other_graph_matrix_path):
    other_matrix=pd.read_csv(other_graph_matrix_path,index_col="source")
    edges_other_matrix=get_edges(other_matrix)
    borders=list(np.arange(0.01,1,0.002))
    for b in borders:
        hamming_dist=calculating_hamming_distance(vectors,b)
        edges_hamming=get_edges(hamming_dist)
        number_equel=0
        for edge in edges_other_matrix:
            if edge in edges_hamming:
                number_equel+=1
            
        percent=number_equel/len(edges_other_matrix)
        if percent>=0.5:
            print(b,hamming_dist)
            break
    
            
        
    
    #     border=np.array([0])
#     res = minimize(percent_equal_edges, border,edges_other_matrix, method='nelder-mead',bounds=(0,1)) #options={'xtol': 1e-8, 'disp': True}
#     print(res.x)

In [166]:
other_graph_matrix_path="Data/adjacency matrices/interv without group and diagnosis/0_intervs_without_group_diagnosis_without_other_hillclimbsearch_bic.csv"

# find_borders_for_optimization_numbers_of_needed_edges(other_graph_matrix_path)
find_border_for_half_edges(other_graph_matrix_path)

Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Aver

Average 0.03954214360041623
Average 0.03954214360041623
Average 0.03954214360041623
Average 0.03954214360041623
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.0437044745057232
Average 0.0437044745057232
Average 0.0437044745057232
Average 0.0437044745057232
Average 0.0437044745057232
Average 0.04578563995837669
Average 0.04578563995837669
Average 0.04786680541103018
Average 0.04786680541103018
Average 0.04786680541103018
Average 0.04994797086368366
Average 0.04994797086368366
Average 0.05202913631633715
Average 0.05202913631633715
Average 0.05202913631633715
Average 0.054110301768990635
Average 0.054110301768990635
Average 0.054110301768990635
Average 0.054110301768990635
Average 0.054110301768990635
Average 0.05619146722164412
Average 0.0582726326742976
Average 0.0582726326742976
Average 0.0582726326742976
Average 0.0582726326742

In [167]:
other_graph_matrix_path="Data/adjacency matrices/interv without group and diagnosis/1_intervs_without_group_diagnosis_without_other_hillclimbsearch_bdeu.csv"

# find_borders_for_optimization_numbers_of_needed_edges(other_graph_matrix_path)
find_border_for_half_edges(other_graph_matrix_path)

Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Aver

Average 0.03954214360041623
Average 0.03954214360041623
Average 0.03954214360041623
Average 0.03954214360041623
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.0437044745057232
Average 0.0437044745057232
Average 0.0437044745057232
Average 0.0437044745057232
Average 0.0437044745057232
Average 0.04578563995837669
Average 0.04578563995837669
Average 0.04786680541103018
Average 0.04786680541103018
Average 0.04786680541103018
Average 0.04994797086368366
Average 0.04994797086368366
Average 0.05202913631633715
Average 0.05202913631633715
Average 0.05202913631633715
Average 0.054110301768990635
Average 0.054110301768990635
Average 0.054110301768990635
Average 0.054110301768990635
Average 0.054110301768990635
Average 0.05619146722164412
Average 0.0582726326742976
Average 0.0582726326742976
Average 0.0582726326742976
Average 0.0582726326742

In [168]:
other_graph_matrix_path="Data/adjacency matrices/interv without group and diagnosis/2_intervs_without_group_diagnosis_without_other_hillclimbsearch_k2.csv"

# find_borders_for_optimization_numbers_of_needed_edges(other_graph_matrix_path)
find_border_for_half_edges(other_graph_matrix_path)

Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Average 0.0
Aver

Average 0.03954214360041623
Average 0.03954214360041623
Average 0.03954214360041623
Average 0.03954214360041623
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.04162330905306972
Average 0.0437044745057232
Average 0.0437044745057232
Average 0.0437044745057232
Average 0.0437044745057232
Average 0.0437044745057232
Average 0.04578563995837669
Average 0.04578563995837669
Average 0.04786680541103018
Average 0.04786680541103018
Average 0.04786680541103018
Average 0.04994797086368366
Average 0.04994797086368366
Average 0.05202913631633715
Average 0.05202913631633715
Average 0.05202913631633715
Average 0.054110301768990635
Average 0.054110301768990635
Average 0.054110301768990635
Average 0.054110301768990635
Average 0.054110301768990635
Average 0.05619146722164412
Average 0.0582726326742976
Average 0.0582726326742976
Average 0.0582726326742976
Average 0.0582726326742

In [185]:
calculating_hamming_distance(df_BN,border=0.1,save_table=True,write_weight=True)

Average 0.09483294384692825


,ICD F17,ICD I10,ICD F60,ICD F32,ICD Z73,ICD Z59,ICD F41,ICD F20,ICD F31,ICD F25,ICD F43,ICD F10,ICD Z56,ICD Z61,ICD F33,ICD Z60,ICD Z63,ICD Z72,social competence:communication training,life skills:training in health awareness and behavior,relaxation techniques:jacobson relaxation,therapy:dealing with psychotic symptoms,everyday competence:building/maintaining daily structure,de-escalation (crisis intervention):n/a,therapy:emotional competence traning,therapy:resource work,life skills:activity development/maintenance,therapy:depression coping,psychoeducation:n/a,n/a:therapy in an individual setting,therapy:cognitive training
ICD F17,0,0.929331,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.907442,0,0,0,0.948093,0,0.947467,0.933709,0,0,0.949343,0,0,0
ICD I10,0.929331,0,0,0,0,0,0,0,0,0.901188,0,0,0,0,0,0,0,0.909318,0,0,0,0.953721,0,0.949343,0.939337,0,0,0.954972,0.904315,0,0
ICD F60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ICD F32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ICD Z73,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.917448,0,0.918074,0.914321,0,0,0.91995,0,0,0
ICD Z59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ICD F41,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ICD F20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ICD F31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.914321,0,0.909944,0,0,0,0.913071,0,0,0
ICD F25,0,0.901188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.931207,0,0.924328,0.913071,0,0,0.927455,0,0,0


Заметки
Делаю функцию
Пробую то что получилось с моей метрикой - лучший border 0.13
Пробую по другому считать расстояние харминга, и попробую мой метод
Также можно констант попробовать, и другие техники

Сделать несколько хемминг графиков и все их также отрисовать, и искать наиболее похожую пару
1. Сделать с весами, и кластеризовать используя веса c весом 0.87
1.25 - Запустить построение БС всеми способами, построить все и визуализировать
1.5 - применить старое расстояние с весами 0.1 и 0.07
2. Сравнять количество ребер весом
3. Для констант монжно пробовать игратся с весом